In [1]:
%load_ext sql
%config SqlMagic.displaylimit = None

displaylimit: Value None will be treated as 0 (no limit)

In [2]:
%sql sqlite:///library.db

Connecting to 'sqlite:///library.db'

In [3]:
%%sql
PRAGMA foreign_keys = ON;


Running query in 'sqlite:///library.db'

++
||
++
++

In [7]:
%%sql
PRAGMA foreign_keys = ON;

Running query in 'sqlite:///library.db'

++
||
++
++

In [ ]:
%%sql
-- Main Tables
            CREATE TABLE IF NOT EXISTS Author (
                AuthorID INTEGER PRIMARY KEY AUTOINCREMENT,
                FirstName TEXT NOT NULL,
                LastName TEXT NOT NULL,
                BirthDate DATE,
                Nationality TEXT,
                Website TEXT
            );

            # In the CREATE TABLE section for Item
            CREATE TABLE IF NOT EXISTS Item (
                ItemID INTEGER PRIMARY KEY AUTOINCREMENT,
                Title TEXT NOT NULL,
                PublicationDate DATE,
                Genre TEXT,
                Type TEXT NOT NULL CHECK (Type IN ('Print Book', 'Online Book', 'Magazine', 'Journal', 'CD', 'Record')),
                BorrowingStatus TEXT DEFAULT 'Available' CHECK (BorrowingStatus IN ('Available', 'Borrowed')),
                Location TEXT,
                URL TEXT,
                ISBN TEXT UNIQUE,
                ReferenceOnly BOOLEAN DEFAULT 0
            );

            CREATE TABLE IF NOT EXISTS Member (
                MemberID INTEGER PRIMARY KEY AUTOINCREMENT,
                FirstName TEXT NOT NULL,
                LastName TEXT NOT NULL,
                ContactInfo TEXT NOT NULL,
                JoinDate DATE NOT NULL DEFAULT CURRENT_DATE,
                Status TEXT DEFAULT 'Active' CHECK (Status IN ('Active', 'Inactive'))
            );

            CREATE TABLE IF NOT EXISTS Borrowing (
                BorrowingID INTEGER PRIMARY KEY AUTOINCREMENT,
                MemberID INTEGER NOT NULL,
                ItemID INTEGER NOT NULL,
                BorrowDate DATE NOT NULL,
                DueDate DATE NOT NULL,
                ReturnDate DATE,
                FineAmount REAL DEFAULT 0.0 CHECK (FineAmount >= 0),
                FOREIGN KEY (MemberID) REFERENCES Member(MemberID) ON DELETE CASCADE,
                FOREIGN KEY (ItemID) REFERENCES Item(ItemID) ON DELETE CASCADE
            );

            CREATE TABLE IF NOT EXISTS Donation (
                DonationID INTEGER PRIMARY KEY AUTOINCREMENT,
                MemberID INTEGER NOT NULL,
                ItemID INTEGER NOT NULL,
                DonationDate DATE NOT NULL DEFAULT CURRENT_DATE,
                Status TEXT DEFAULT 'Pending' CHECK (Status IN ('Pending', 'Accepted', 'Rejected')),
                FOREIGN KEY (MemberID) REFERENCES Member(MemberID) ON DELETE CASCADE,
                FOREIGN KEY (ItemID) REFERENCES Item(ItemID) ON DELETE CASCADE
            );

            CREATE TABLE IF NOT EXISTS Staff (
                StaffID INTEGER PRIMARY KEY AUTOINCREMENT,
                FirstName TEXT NOT NULL,
                LastName TEXT NOT NULL,
                ContactInfo TEXT NOT NULL,
                Position TEXT NOT NULL,
                Salary REAL CHECK (Salary >= 0),
                HireDate DATE NOT NULL DEFAULT CURRENT_DATE
            );

            CREATE TABLE IF NOT EXISTS SocialRoom (
                SocialRoomID INTEGER PRIMARY KEY AUTOINCREMENT,
                RoomName TEXT NOT NULL,
                Capacity INTEGER NOT NULL CHECK (Capacity > 0)
            );

            CREATE TABLE IF NOT EXISTS Event (
                EventID INTEGER PRIMARY KEY AUTOINCREMENT,
                Name TEXT NOT NULL,
                Description TEXT,
                Date DATE NOT NULL,
                Time TEXT NOT NULL,
                RecommendedAudience TEXT,
                SocialRoomID INTEGER NOT NULL,
                FOREIGN KEY (SocialRoomID) REFERENCES SocialRoom(SocialRoomID) ON DELETE RESTRICT
            );

            CREATE TABLE IF NOT EXISTS FutureItem (
                FutureItemID INTEGER PRIMARY KEY AUTOINCREMENT,
                Title TEXT NOT NULL,
                ExpectedArrivalDate DATE,
                Status TEXT DEFAULT 'Pending' CHECK (Status IN ('Pending', 'Arrived', 'Cancelled'))
            );

            -- Junction Tables
            CREATE TABLE IF NOT EXISTS Item_Author (
                ItemID INTEGER NOT NULL,
                AuthorID INTEGER NOT NULL,
                PRIMARY KEY (ItemID, AuthorID),
                FOREIGN KEY (ItemID) REFERENCES Item(ItemID) ON DELETE CASCADE,
                FOREIGN KEY (AuthorID) REFERENCES Author(AuthorID) ON DELETE CASCADE
            );

            CREATE TABLE IF NOT EXISTS Author_FutureItem (
                AuthorID INTEGER NOT NULL,
                FutureItemID INTEGER NOT NULL,
                PRIMARY KEY (AuthorID, FutureItemID),
                FOREIGN KEY (AuthorID) REFERENCES Author(AuthorID) ON DELETE CASCADE,
                FOREIGN KEY (FutureItemID) REFERENCES FutureItem(FutureItemID) ON DELETE CASCADE
            );

            CREATE TABLE IF NOT EXISTS Event_Member (
                EventID INTEGER NOT NULL,
                MemberID INTEGER NOT NULL,
                RegistrationDate DATE NOT NULL,
                PRIMARY KEY (EventID, MemberID),
                FOREIGN KEY (EventID) REFERENCES Event(EventID) ON DELETE CASCADE,
                FOREIGN KEY (MemberID) REFERENCES Member(MemberID) ON DELETE CASCADE
            );

            CREATE TABLE IF NOT EXISTS Event_Member_Volunteer (
                EventID INTEGER NOT NULL,
                MemberID INTEGER NOT NULL,
                VolunteerRole TEXT NOT NULL,
                VolunteerDate DATE NOT NULL,
                PRIMARY KEY (EventID, MemberID),
                FOREIGN KEY (EventID) REFERENCES Event(EventID) ON DELETE CASCADE,
                FOREIGN KEY (MemberID) REFERENCES Member(MemberID) ON DELETE CASCADE
            );

            CREATE TABLE IF NOT EXISTS Event_Staff (
                EventID INTEGER NOT NULL,
                StaffID INTEGER NOT NULL,
                PRIMARY KEY (EventID, StaffID),
                FOREIGN KEY (EventID) REFERENCES Event(EventID) ON DELETE CASCADE,
                FOREIGN KEY (StaffID) REFERENCES Staff(StaffID) ON DELETE CASCADE
            );
                          
            CREATE TABLE IF NOT EXISTS Fine (
                MemberID INTEGER PRIMARY KEY,
                TotalFine REAL DEFAULT 0.0 CHECK (TotalFine >= 0),
                FOREIGN KEY (MemberID) REFERENCES Member(MemberID) ON DELETE CASCADE
            );

            -- Triggers
            CREATE TRIGGER IF NOT EXISTS UpdateBorrowingStatusOnBorrow
            AFTER INSERT ON Borrowing
            FOR EACH ROW
            WHEN NEW.ReturnDate IS NULL
            BEGIN
                UPDATE Item
                SET BorrowingStatus = 'Borrowed'
                WHERE ItemID = NEW.ItemID;
            END;

            CREATE TRIGGER IF NOT EXISTS UpdateBorrowingStatusOnReturn
            AFTER UPDATE OF ReturnDate ON Borrowing
            FOR EACH ROW
            WHEN NEW.ReturnDate IS NOT NULL AND OLD.ReturnDate IS NULL
            BEGIN
                UPDATE Item
                SET BorrowingStatus = 'Available'
                WHERE ItemID = NEW.ItemID;
            END;

            CREATE TRIGGER IF NOT EXISTS CalculateFineOnReturn
            AFTER UPDATE OF ReturnDate ON Borrowing
            FOR EACH ROW
            WHEN NEW.ReturnDate IS NOT NULL AND NEW.ReturnDate > NEW.DueDate
            BEGIN
                UPDATE Borrowing
                SET FineAmount = (JULIANDAY(NEW.ReturnDate) - JULIANDAY(NEW.DueDate)) * 0.50
                WHERE BorrowingID = NEW.BorrowingID;

                INSERT INTO Fine (MemberID, TotalFine)
                VALUES (NEW.MemberID, (JULIANDAY(NEW.ReturnDate) - JULIANDAY(NEW.DueDate)) * 0.50)
                ON CONFLICT(MemberID)
                DO UPDATE SET TotalFine = TotalFine + (JULIANDAY(NEW.ReturnDate) - JULIANDAY(NEW.DueDate)) * 0.50;
            END;


Running query in 'sqlite:///library.db'

++
||
++
++

In [ ]:
%%sql
-- Author
INSERT INTO Author (FirstName, LastName, BirthDate, Nationality, Website) VALUES
('Jane', 'Austen', '1775-12-16', 'British', NULL),
('George', 'Orwell', '1903-06-25', 'British', NULL),
('J.K.', 'Rowling', '1965-07-31', 'British', 'www.jkrowling.com'),
('Stephen', 'King', '1947-09-21', 'American', 'www.stephenking.com'),
('Agatha', 'Christie', '1890-09-15', 'British', NULL),
('Mark',' Twain', '1835-11-30', 'American', NULL),
('Toni', 'Morrison', '1931-02-18', 'American', NULL),
('Haruki', 'Murakami', '1949-01-12', 'Japanese', 'www.harukimurakami.com'),
('Gabriel García', 'Márquez', '1927-03-06', 'Colombian', NULL),
('Chimamanda Ngozi', 'Adichie', '1977-09-15', 'Nigerian', 'www.chimamanda.com'),
('Neil', 'Gaiman', '1960-11-10', 'British', 'www.neilgaiman.com'),
('Margaret' ,'Atwood', '1939-11-18', 'Canadian', 'www.margaretatwood.ca'),
('Brandon', 'Sanderson', '1975-12-19', 'American', 'www.brandonsanderson.com'),
('Yuval Noah', 'Harari', '1976-02-24', 'Israeli', 'www.ynharari.com'),
('Isabel', 'Allende', '1942-08-02', 'Chilean', 'www.isabelallende.com');

-- Item
# In the INSERT INTO Item section
INSERT INTO Item (Title, PublicationDate, Genre, Type, BorrowingStatus, Location, URL, ISBN, ReferenceOnly) VALUES
('Pride and Prejudice', '1813-01-28', 'Romance', 'Print Book', 'Available', 'Shelf A1', NULL, '9080141439518', 0),
('1984', '1949-06-08', 'Dystopia', 'Print Book', 'Available', 'Shelf B2', NULL, '9780451520935', 0),
('Harry Potter and the Sorcerer''s Stone', '1997-06-26', 'Fantasy', 'Print Book', 'Borrowed', 'Shelf C3', NULL, '9480590353427', 0),
('The Shining', '1977-01-28', 'Horror', 'Print Book', 'Available', 'Shelf D4', NULL, '9780385021675', 0),
('Murder on the Orient Express', '1934-01-01', 'Mystery', 'Print Book', 'Available', 'Shelf E5', NULL, '9780062093662', 0),
('Science Weekly', '2025-03-01', 'Science', 'Magazine', 'Available', 'Rack M1', NULL, NULL, 1), -- Reference-only magazine
('Jazz Classics', '1990-05-15', 'Music', 'CD', 'Available', 'Rack CD1', NULL, NULL, 0),
('Norwegian Wood', '1987-08-04', 'Fiction', 'Online Book', 'Available', NULL, 'http://example.com/nw', NULL, 0),
('One Hundred Years of Solitude', '1967-05-30', 'Magical Realism', 'Print Book', 'Borrowed', 'Shelf F6', NULL, '9700060883287', 0),
('Americanah', '2013-05-14', 'Fiction', 'Print Book', 'Available', 'Shelf G7', NULL, '9780307055925', 0),
('American Gods', '2001-06-19', 'Fantasy', 'Print Book', 'Available', 'Shelf H1', NULL, '9780060058123', 0),
('The Handmaid''s Tale', '1985-08-17', 'Dystopian', 'Print Book', 'Available', 'Shelf H2', NULL, '9780385490818', 0),
('Mistborn', '2006-07-17', 'Fantasy', 'Print Book', 'Available', 'Shelf H3', NULL, '9780765311708', 0),
('Sapiens: A Brief History of Humankind', '2011-01-01', 'History', 'Print Book', 'Available', 'Shelf H4', NULL, '9780042316097', 1), -- Reference-only book
('The House of the Spirits', '1982-03-01', 'Magical Realism', 'Print Book', 'Available', 'Shelf H5', NULL, '9780753383805', 0);

-- Member
INSERT INTO Member (FirstName, LastName, ContactInfo, JoinDate, Status) VALUES
('Alice', 'Smith', 'alice@example.com', '2023-01-15', 'Active'),
('Bob', 'Johnson', 'bob@example.com', '2023-02-20', 'Active'),
('Clara', 'Lee', 'clara@example.com', '2023-03-10', 'Inactive'),
('David', 'Kim', 'david@example.com', '2023-04-05', 'Active'),
('Emma', 'Brown', 'emma@example.com', '2023-05-12', 'Active'),
('Frank', 'White', 'frank@example.com', '2023-06-18', 'Active'),
('Grace', 'Chen', 'grace@example.com', '2023-07-22', 'Active'),
('Henry', 'Davis', 'henry@example.com', '2023-08-30', 'Active'),
('Isabel', 'Garcia', 'isabel@example.com', '2023-09-15', 'Active'),
('James', 'Patel', 'james@example.com', '2023-10-01', 'Active'),
('Natalie', 'Reed', 'natalie@example.com', '2023-11-01', 'Active'),
('Carlos', 'Mendez', 'carlos@example.com', '2023-11-02', 'Active'),
('Luna', 'Zhao', 'luna@example.com', '2023-11-03', 'Active'),
('Victor', 'O''Brien', 'victor@example.com', '2023-11-04', 'Active'),
('Nina', 'Dubois', 'nina@example.com', '2023-11-05', 'Active');

-- Borrowing
INSERT INTO Borrowing (MemberID, ItemID, BorrowDate, DueDate, ReturnDate, FineAmount) VALUES
(1, 3, '2025-03-01', '2025-03-15', NULL, 0.0), -- Harry Potter borrowed
(2, 9, '2025-03-05', '2025-03-19', NULL, 0.0), -- One Hundred Years borrowed
(3, 1, '2025-02-10', '2025-02-24', '2025-03-01', 3.5), -- Pride and Prejudice returned late
(4, 2, '2025-03-10', '2025-03-24', '2025-03-20', 0.0), -- 1984 returned on time
(5, 4, '2025-03-12', '2025-03-26', NULL, 0.0), -- The Shining borrowed
(6, 5, '2025-03-15', '2025-03-29', NULL, 0.0), -- Murder on the Orient Express borrowed
(7, 6, '2025-03-18', '2025-04-01', NULL, 0.0), -- Science Weekly borrowed
(8, 7, '2025-03-20', '2025-04-03', NULL, 0.0), -- Jazz Classics borrowed
(9, 8, '2025-03-22', '2025-04-05', NULL, 0.0), -- Norwegian Wood borrowed
(10, 10, '2025-03-23', '2025-04-06', NULL, 0.0), -- Americanah borrowed
(11, 11, '2025-04-02', '2025-04-16', NULL, 0.0), -- American Gods borrowed by Natalie Reed,
(12, 12, '2025-04-02', '2025-04-16', NULL, 0.0), -- The Handmaid's Tale borrowed by Carlos Mendez,
(13, 13, '2025-04-02', '2025-04-16', NULL, 0.0), -- Mistborn borrowed by Luna Zhao,
(14, 14, '2025-04-02', '2025-04-16', NULL, 0.0), -- Sapiens borrowed by Victor O'Brien,
(15, 15, '2025-04-02', '2025-04-16', NULL, 0.0); -- The House of the Spirits borrowed by Nina Dubois;


-- Donation
INSERT INTO Donation (MemberID, ItemID, DonationDate, Status) VALUES
(1, 1, '2025-01-10', 'Accepted'),
(2, 2, '2025-01-15', 'Accepted'),
(3, 3, '2025-01-20', 'Accepted'),
(4, 4, '2025-01-25', 'Accepted'),
(5, 5, '2025-02-01', 'Accepted'),
(6, 6, '2025-02-05', 'Accepted'),
(7, 7, '2025-02-10', 'Accepted'),
(8, 8, '2025-02-15', 'Accepted'),
(9, 9, '2025-02-20', 'Accepted'),
(10, 10, '2025-02-25', 'Accepted'),
(11, 11, '2025-04-02', 'Accepted'),
(12, 12, '2025-04-02', 'Accepted'),
(13, 13, '2025-04-02', 'Accepted'),
(14, 14, '2025-04-02', 'Accepted'),
(15, 15, '2025-04-02', 'Accepted');

-- Staff
INSERT INTO Staff (FirstName, LastName, ContactInfo, Position, Salary, HireDate) VALUES
('Liam', 'Carter', 'liam@example.com', 'Librarian', 50000, '2020-01-01'),
('Olivia', 'Nguyen', 'olivia@example.com', 'Assistant Librarian', 40000, '2021-03-15'),
('Noah', 'Patel', 'noah@example.com', 'Event Coordinator', 45000, '2022-05-10'),
('Sophia', 'Kim', 'sophia@example.com', 'Clerk', 35000, '2023-07-20'),
('Ethan', 'Brown', 'ethan@example.com', 'IT Specialist', 55000, '2021-09-01'),
('Ava', 'Lopez', 'ava@example.com', 'Librarian', 52000, '2020-11-15'),
('Mason', 'Lee', 'mason@example.com', 'Clerk', 34000, '2023-02-28'),
('Isabella', 'Wang', 'isabella@example.com', 'Assistant Librarian', 41000, '2022-04-10'),
('Lucas', 'Green', 'lucas@example.com', 'Security', 38000, '2021-06-15'),
('Mia', 'Adams', 'mia@example.com', 'Event Coordinator', 46000, '2023-08-05'),
('Amelia', 'Stone', 'amelia@library.com', 'Archivist', 48000, '2024-01-15'),
('Leo', 'Martinez', 'leo@library.com', 'IT Assistant', 42000, '2024-02-10'),
('Nora', 'Black', 'nora@library.com', 'Librarian', 53000, '2023-12-05'),
('Omar', 'Hussein', 'omar@library.com', 'Security', 39000, '2024-03-01'),
('Jade', 'Nguyen', 'jade@library.com', 'Event Manager', 51000, '2024-01-25');

-- SocialRoom
INSERT INTO SocialRoom (RoomName, Capacity) VALUES
('Reading Room A', 30),
('Media Room B', 20),
('Event Hall C', 50),
('Study Room D', 15),
('Art Gallery E', 40),
('Conference Room F', 25),
('Music Room G', 20),
('Film Room H', 35),
('Book Club Room I', 15),
('Workshop Room J', 30),
('Quiet Study Zone', 12),
('Digital Lab', 18),
('Lecture Hall Z', 60),
('Reading Nook', 10),
('Creative Writing Room', 20);

-- Event
INSERT INTO Event (Name, Description, Date, Time, RecommendedAudience, SocialRoomID) VALUES
('Book Club: Classics', 'Discussing Pride and Prejudice', '2025-04-01', '18:00', 'Adults', 1),
('Film Screening: 1984', 'Movie adaptation', '2025-04-05', '19:00', 'Teens and Adults', 8),
('Art Show: Local Artists', 'Exhibit of local art', '2025-04-10', '14:00', 'All Ages', 5),
('Science Talk', 'Latest discoveries', '2025-04-15', '17:00', 'Adults', 6),
('Kids Story Time', 'Reading Harry Potter', '2025-04-20', '10:00', 'Children', 4),
('Music Night', 'Jazz classics', '2025-04-25', '20:00', 'Adults', 7),
('Writing Workshop', 'Creative writing tips', '2025-04-30', '15:00', 'Teens and Adults', 10),
('Mystery Book Club', 'Agatha Christie focus', '2025-05-05', '18:30', 'Adults', 9),
('Tech Seminar', 'Digital libraries', '2025-05-10', '13:00', 'Adults', 3),
('Poetry Reading', 'Local poets', '2025-05-15', '19:00', 'All Ages', 2),
('Meet the Author: Neil Gaiman', 'Q&A with Neil Gaiman', '2025-06-01', '17:00', 'Adults', 11),
('Dystopia Discussion', 'Exploring The Handmaid''s Tale', '2025-06-05', '18:30', 'Teens and Adults', 12),
('History Unplugged', 'Lecture on Sapiens by Harari', '2025-06-10', '16:00', 'Adults', 13),
('Fantasy Writers Panel', 'Panel with fantasy authors', '2025-06-15', '14:00', 'All Ages', 14),
('Magical Realism Workshop', 'Guided writing based on Allende', '2025-06-20', '11:00', 'Adults', 15);


-- FutureItem
INSERT INTO FutureItem (Title, ExpectedArrivalDate, Status) VALUES
('The Testaments', '2025-06-01', 'Pending'),
('Dune Messiah', '2025-06-15', 'Pending'),
('The Overstory', '2025-07-01', 'Pending'),
('Circe', '2025-07-15', 'Pending'),
('Klara and the Sun', '2025-08-01', 'Pending'),
('Project Hail Mary', '2025-08-15', 'Pending'),
('The Midnight Library', '2025-09-01', 'Pending'),
('Anxious People', '2025-09-15', 'Pending'),
('The Invisible Life of Addie LaRue', '2025-10-01', 'Pending'),
('Cloud Cuckoo Land', '2025-10-15', 'Pending'),
('The Ocean at the End of the Lane', '2025-11-01', 'Pending'),
('Oryx and Crake', '2025-11-15', 'Pending'),
('The Way of Kings', '2025-12-01', 'Pending'),
('Homo Deus', '2025-12-15', 'Pending'),
('Paula', '2026-01-01', 'Pending');

-- Item_Author
INSERT INTO Item_Author (ItemID, AuthorID) VALUES
(1, 1), -- Pride and Prejudice by Jane Austen
(2, 2), -- 1984 by George Orwell
(3, 3), -- Harry Potter by J.K. Rowling
(4, 4), -- The Shining by Stephen King
(5, 5), -- Murder on the Orient Express by Agatha Christie
(8, 8), -- Norwegian Wood by Haruki Murakami
(9, 9), -- One Hundred Years by Gabriel García Márquez
(10, 10), -- Americanah by Chimamanda Ngozi Adichie
(6, 7), -- Science Weekly by Toni Morrison (fictional)
(7, 6), -- Jazz Classics by Mark Twain (fictional)
(11, 11), -- American Gods by Neil Gaiman,
(12, 12), -- The Handmaid's Tale by Margaret Atwood,
(13, 13), -- Mistborn by Brandon Sanderson,
(14, 14), -- Sapiens: A Brief History of Humankind by Yuval Noah Harari,
(15, 15); -- The House of the Spirits by Isabel Allende;

-- Author_FutureItem
INSERT INTO Author_FutureItem (AuthorID, FutureItemID) VALUES
(3, 1), -- J.K. Rowling - The Testaments
(2, 2), -- George Orwell - Dune Messiah
(4, 3), -- Stephen King - The Overstory
(8, 4), -- Haruki Murakami - Circe
(10, 5), -- Chimamanda Ngozi Adichie - Klara and the Sun
(1, 6), -- Jane Austen - Project Hail Mary
(5, 7), -- Agatha Christie - The Midnight Library
(6, 8), -- Mark Twain - Anxious People
(7, 9), -- Toni Morrison - The Invisible Life
(9, 10), -- Gabriel García Márquez - Cloud Cuckoo Land
(11, 11), -- Neil Gaiman - The Ocean at the End of the Lane,
(12, 12), -- Margaret Atwood - Oryx and Crake,
(13, 13), -- Brandon Sanderson - The Way of Kings,
(14, 14), -- Yuval Noah Harari - Homo Deus,
(15, 15); -- Isabel Allende - Paula;

-- Event_Member
INSERT INTO Event_Member (EventID, MemberID, RegistrationDate) VALUES
(1, 1, '2025-03-20'),
(1, 2, '2025-03-21'),
(2, 3, '2025-03-22'),
(3, 4, '2025-03-23'),
(4, 5, '2025-03-24'),
(5, 6, '2025-03-25'),
(6, 7, '2025-03-26'),
(7, 8, '2025-03-27'),
(8, 9, '2025-03-28'),
(9, 10, '2025-03-29'),
(11, 11, '2025-04-02'), 
(12, 12, '2025-04-02'), 
(13, 13, '2025-04-02'), 
(14, 14, '2025-04-02'), 
(15, 15, '2025-04-02');

-- Event_Member_Volunteer
INSERT INTO Event_Member_Volunteer (EventID, MemberID, VolunteerRole, VolunteerDate) VALUES
(1, 3, 'Moderator', '2025-04-01'),
(2, 4, 'Usher', '2025-04-05'),
(3, 5, 'Setup Crew', '2025-04-10'),
(4, 6, 'Speaker Assistant', '2025-04-15'),
(5, 7, 'Reader', '2025-04-20'),
(6, 8, 'Sound Tech', '2025-04-25'),
(7, 9, 'Facilitator', '2025-04-30'),
(8, 10, 'Moderator', '2025-05-05'),
(9, 1, 'Tech Support', '2025-05-10'),
(10, 2, 'Greeter', '2025-05-15'),
(11, 11, 'Host', '2025-04-02'), -- Member 11 volunteered as Host for Event 11,
(12, 12, 'Speaker', '2025-04-02'), -- Member 12 volunteered as Speaker for Event 12,
(13, 13, 'Greeter', '2025-04-02'), -- Member 13 volunteered as Greeter for Event 13,
(14, 14, 'Tech Assistant', '2025-04-02'), -- Member 14 volunteered as Tech Assistant for Event 14,
(15, 15, 'Photographer', '2025-04-02'); -- Member 15 volunteered as Photographer for Event 15;

-- Event_Staff
INSERT INTO Event_Staff (EventID, StaffID) VALUES
(1, 1),
(2, 2),
(3, 3),
(4, 4),
(5, 5),
(6, 6),
(7, 7),
(8, 8),
(9, 9),
(10, 10),
(11, 11),
(12, 12), 
(13, 13), 
(14, 14), 
(15, 15); 


Running query in 'sqlite:///library.db'

15 rows affected.

15 rows affected.

15 rows affected.

15 rows affected.

...

15 rows affected.

15 rows affected.

15 rows affected.

15 rows affected.

++
||
++
++